# Practica 07: Preparacion de Datos para su Análisis (ETL) Base de datos y Api

**Aignatura:** Extarcción de Conocimientos en Bases de Datos
<br>Unidad 2: Preparación de Datos
<br>PE: Ingenieria Desarrollo y Gestion de Software

Realizado por: **Irving Morales** - 220732

# Ejercicios de Carga

### Ejercicio 1: Datos de BD

Importar datos de una BD SQL

In [1]:
!pip install sqlalchemy pymysql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(f"credenciales D:")

In [4]:
df_personas = pd.read_sql("SELECT * FROM tbb_personas", engine)
df_personas.head()  


,id,titulo,nombre,primer_apellido,segundo_apellido,curp,genero,grupo_sanguineo,fecha_nacimiento,estatus,fecha_registro,fecha_actualizacion
0,09057c00-0ff8-11f0-b70d-3c557613b8e0,None,Fernanda,Castillo,Gutiérrez,CSGF880622FN61,F,O+,1988-06-22,1,2025-04-02 13:24:02,None
1,125e9a19-0ff8-11f0-b70d-3c557613b8e0,None,Alex,Rojas,Delgado,RSDA781208N/BO68,N/B,A+,1978-12-08,1,2025-04-02 13:24:18,None
2,26f93a89-11a9-11f0-b70d-3c557613b8e0,Lic.,Juan,Rodríguez,Ramírez,RRRJ980929MD63,M,A+,1998-09-29,1,2025-04-04 17:04:24,None
3,5253f56b-0ff8-11f0-b70d-3c557613b8e0,None,Andrea,Torres,Gutiérrez,TTGA970301FK45,F,B+,1997-03-01,1,2025-04-02 13:26:05,None
4,bdb8af10-11a9-11f0-b70d-3c557613b8e0,None,Miguel,Ramírez,Hernández,RMHM070303MT27,M,B+,2007-03-03,1,2025-04-04 17:08:37,None


In [5]:
df_personas.size

3684

In [6]:
print(len(df_personas))

307


In [7]:
df_personas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   307 non-null    object        
 1   titulo               269 non-null    object        
 2   nombre               307 non-null    object        
 3   primer_apellido      307 non-null    object        
 4   segundo_apellido     307 non-null    object        
 5   curp                 307 non-null    object        
 6   genero               307 non-null    object        
 7   grupo_sanguineo      307 non-null    object        
 8   fecha_nacimiento     307 non-null    object        
 9   estatus              307 non-null    int64         
 10  fecha_registro       307 non-null    datetime64[ns]
 11  fecha_actualizacion  0 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 28.9+ KB


In [10]:
df_personas = df_personas.dropna(subset=['titulo'])

In [11]:
print(len(df_personas))

269


In [12]:
df_personas = df_personas[['titulo', 'nombre', 'estatus', 'genero']]

In [14]:
print(len(df_personas))
df_personas

269


,titulo,nombre,estatus,genero
2,Lic.,Juan,1,M
7,Lic.,Ricardo,1,M
8,Lic.,Chris,1,N/B
9,Dr.,Casey,1,N/B
10,Dr.,Javier,1,M
...,...,...,...,...
300,Lic.,Ricardo,1,M
301,Ing.,Juan,1,M
302,Ing.,Gabriela,1,F
303,Lic.,Javier,1,M


In [15]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TEMPORARY TABLE tmp_personas_filtradas (
            titulo VARCHAR(60),
            nombre VARCHAR(60),
            estatus TINYINT(1),
            genero ENUM('M', 'F', 'N/B')
        );
    """))


In [16]:
df_personas.to_sql(
    name='tmp_personas_filtradas',
    con=engine,
    if_exists='append',  # usamos append porque la tabla ya existe
    index=False
)


269

In [18]:
verification_query = f"SELECT * FROM tmp_personas_filtradas LIMIT 5"
result = pd.read_sql(verification_query, engine)
print(result)

  titulo   nombre  estatus genero
0   Lic.     Juan        1      M
1   Lic.  Ricardo        1      M
2   Lic.    Chris        1    N/B
3    Dr.    Casey        1    N/B
4    Dr.   Javier        1      M


### Ejercicio 2: Datos de API

Importar datos de una API 

In [21]:
import requests
import pandas as pd

url = "https://swapi.py4e.com/api/people"
all_people = []

while url:                     
    data = requests.get(url).json()
    all_people.extend(data["results"])
    url = data["next"]        
    
def get_planet_name(url):
    return requests.get(url).json().get("name", "Desconocido")
    
df_api = pd.json_normalize(all_people)
df_api['homeworld_name'] = df_api['homeworld'].apply(get_planet_name)

In [22]:
df_api

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,homeworld_name
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],"[https://swapi.py4e.com/api/vehicles/14/, http...","[https://swapi.py4e.com/api/starships/12/, htt...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.py4e.com/api/people/1/,Tatooine
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.py4e.com/api/people/2/,Tatooine
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.py4e.com/api/planets/8/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.py4e.com/api/people/3/,Naboo
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],[],[https://swapi.py4e.com/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.py4e.com/api/people/4/,Tatooine
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.py4e.com/api/planets/2/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],[https://swapi.py4e.com/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.py4e.com/api/people/5/,Alderaan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Finn,unknown,unknown,black,dark,dark,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[],2015-04-17T06:52:40.793621Z,2015-04-17T06:52:40.793674Z,https://swapi.py4e.com/api/people/84/,unknown
83,Rey,unknown,unknown,brown,light,hazel,unknown,female,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[],2015-04-17T06:54:01.495077Z,2015-04-17T06:54:01.495128Z,https://swapi.py4e.com/api/people/85/,unknown
84,Poe Dameron,unknown,unknown,brown,light,brown,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[https://swapi.py4e.com/api/starships/77/],2015-04-17T06:55:21.622786Z,2015-04-17T06:55:21.622835Z,https://swapi.py4e.com/api/people/86/,unknown
85,BB8,none,unknown,none,none,black,unknown,none,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/2/],[],[],2015-04-17T06:57:38.061346Z,2015-04-17T06:57:38.061453Z,https://swapi.py4e.com/api/people/87/,unknown


In [23]:
df_api.dropna()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,homeworld_name
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],"[https://swapi.py4e.com/api/vehicles/14/, http...","[https://swapi.py4e.com/api/starships/12/, htt...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.py4e.com/api/people/1/,Tatooine
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.py4e.com/api/people/2/,Tatooine
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.py4e.com/api/planets/8/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.py4e.com/api/people/3/,Naboo
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.py4e.com/api/planets/1/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],[],[https://swapi.py4e.com/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.py4e.com/api/people/4/,Tatooine
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.py4e.com/api/planets/2/,"[https://swapi.py4e.com/api/films/1/, https://...",[https://swapi.py4e.com/api/species/1/],[https://swapi.py4e.com/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.py4e.com/api/people/5/,Alderaan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Finn,unknown,unknown,black,dark,dark,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[],2015-04-17T06:52:40.793621Z,2015-04-17T06:52:40.793674Z,https://swapi.py4e.com/api/people/84/,unknown
83,Rey,unknown,unknown,brown,light,hazel,unknown,female,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[],2015-04-17T06:54:01.495077Z,2015-04-17T06:54:01.495128Z,https://swapi.py4e.com/api/people/85/,unknown
84,Poe Dameron,unknown,unknown,brown,light,brown,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/1/],[],[https://swapi.py4e.com/api/starships/77/],2015-04-17T06:55:21.622786Z,2015-04-17T06:55:21.622835Z,https://swapi.py4e.com/api/people/86/,unknown
85,BB8,none,unknown,none,none,black,unknown,none,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/films/7/],[https://swapi.py4e.com/api/species/2/],[],[],2015-04-17T06:57:38.061346Z,2015-04-17T06:57:38.061453Z,https://swapi.py4e.com/api/people/87/,unknown


In [24]:
df_api = df_api[['name', 'birth_year', 'gender', 'homeworld_name']]

In [25]:
df_api

,name,birth_year,gender,homeworld_name
0,Luke Skywalker,19BBY,male,Tatooine
1,C-3PO,112BBY,n/a,Tatooine
2,R2-D2,33BBY,n/a,Naboo
3,Darth Vader,41.9BBY,male,Tatooine
4,Leia Organa,19BBY,female,Alderaan
...,...,...,...,...
82,Finn,unknown,male,unknown
83,Rey,unknown,female,unknown
84,Poe Dameron,unknown,male,unknown
85,BB8,unknown,none,unknown


In [26]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TEMPORARY TABLE tmp_sw_personajes (
            name VARCHAR(100),
            birth_year VARCHAR(20),
            gender VARCHAR(20),
            homeworld_name VARCHAR(100)
        );
    """))


In [27]:
df_api.to_sql(
    name='tmp_sw_personajes',
    con=engine,
    if_exists='append', 
    index=False
)

87

In [28]:
verification_query = f"SELECT * FROM tmp_sw_personajes LIMIT 35"
result = pd.read_sql(verification_query, engine)
print(result)

                     name birth_year         gender  homeworld_name
0          Luke Skywalker      19BBY           male        Tatooine
1                   C-3PO     112BBY            n/a        Tatooine
2                   R2-D2      33BBY            n/a           Naboo
3             Darth Vader    41.9BBY           male        Tatooine
4             Leia Organa      19BBY         female        Alderaan
5               Owen Lars      52BBY           male        Tatooine
6      Beru Whitesun lars      47BBY         female        Tatooine
7                   R5-D4    unknown            n/a        Tatooine
8       Biggs Darklighter      24BBY           male        Tatooine
9          Obi-Wan Kenobi      57BBY           male         Stewjon
10       Anakin Skywalker    41.9BBY           male        Tatooine
11         Wilhuff Tarkin      64BBY           male          Eriadu
12              Chewbacca     200BBY           male        Kashyyyk
13               Han Solo      29BBY           m